<a href="https://colab.research.google.com/github/nghia17420/pose-estimation-project-2025/blob/main/Pose_Estimation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

!pip install numpy==1.26.4
!pip install mediapipe==0.10.21 opencv-python==4.11.0.86 tensorflow==2.19.0


In [11]:
import cv2
import mediapipe as mp
import numpy as np
import time
import csv
from datetime import datetime
from collections import deque
import json
import os

In [12]:
class DeadliftConfig:
    """Configuration parameters for deadlift analysis"""

    # MediaPipe Configuration
    MIN_DETECTION_CONFIDENCE = 0.5
    MIN_TRACKING_CONFIDENCE = 0.5
    MODEL_COMPLEXITY = 1  # 0=Lite, 1=Full, 2=Heavy (use 0 for Raspberry Pi)

    # Biomechanical Safety Thresholds (in Newtons)
    COMPRESSION_SAFE_LIMIT = 3433  # NIOSH 1981
    COMPRESSION_MAX_LIMIT = 6400   # Maximum permissible
    SHEAR_SAFE_LIMIT = 500         # University of Waterloo
    SHEAR_MAX_LIMIT = 1000         # Maximum permissible

    # Angle Thresholds for Form Analysis (in degrees)
    TRUNK_ANGLE_MIN = 15   # Minimum trunk angle at start
    TRUNK_ANGLE_MAX = 45   # Maximum trunk angle at start
    KNEE_ANGLE_MIN = 60    # Minimum knee flexion
    KNEE_ANGLE_MAX = 110   # Maximum knee flexion
    HIP_ANGLE_MIN = 60     # Minimum hip flexion

    # Stage Detection Thresholds
    SETUP_HIP_ANGLE = 80
    LIFTING_HIP_ANGLE = 140
    LOCKOUT_HIP_ANGLE = 165

    # Video Configuration
    FRAME_WIDTH = 640
    FRAME_HEIGHT = 480
    FPS_TARGET = 30

    # Data Collection
    SAVE_VIDEO = True
    SAVE_LANDMARKS = True
    OUTPUT_DIR = "deadlift_data"

    # MediaPipe Landmark Indices
    LANDMARKS = {
        'nose': 0,
        'left_shoulder': 11,
        'right_shoulder': 12,
        'left_elbow': 13,
        'right_elbow': 14,
        'left_wrist': 15,
        'right_wrist': 16,
        'left_hip': 23,
        'right_hip': 24,
        'left_knee': 25,
        'right_knee': 26,
        'left_ankle': 27,
        'right_ankle': 28,
        'left_heel': 29,
        'right_heel': 30,
        'left_foot': 31,
        'right_foot': 32
    }

# ============================================================================
# BIOMECHANICAL CALCULATOR CLASS
# ============================================================================
class BiomechanicalCalculator:
    """
    Implements Chaffin's biomechanical model for calculating
    compression and shear forces on L5/S1 disc during deadlifts
    """

    def __init__(self, body_weight_kg=70, load_weight_kg=20, height_m=1.75):
        """
        Initialize calculator with user's body parameters

        Args:
            body_weight_kg: User's body weight in kilograms
            load_weight_kg: Weight of barbell/load in kilograms
            height_m: User's height in meters
        """
        self.body_weight = body_weight_kg
        self.load_weight = load_weight_kg
        self.height = height_m

        # Calculate body segment weights (as fractions of body weight)
        self.trunk_weight = self.body_weight * 0.497  # Head, arms, trunk
        self.load_force = self.load_weight * 9.81     # Convert to Newtons
        self.trunk_force = self.trunk_weight * 9.81   # Convert to Newtons

        # Erector spinae muscle parameters
        self.alpha = 12  # Muscle angle with horizontal (degrees)
        self.e = 0.05    # Distance from L5/S1 to muscle (meters)

    def calculate_distance(self, point1, point2):
        """Calculate Euclidean distance between two 3D points"""
        return np.sqrt(
            (point1[0] - point2[0])**2 +
            (point1[1] - point2[1])**2 +
            (point1[2] - point2[2])**2
        )

    def calculate_angle_3d(self, point1, point2, point3):
        """
        Calculate angle at point2 formed by point1-point2-point3

        Args:
            point1, point2, point3: 3D coordinates [x, y, z]

        Returns:
            Angle in degrees
        """
        a = np.array(point1)
        b = np.array(point2)
        c = np.array(point3)

        ba = a - b
        bc = c - b

        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        angle = np.arccos(cosine_angle)

        return np.degrees(angle)

    def calculate_trunk_angle(self, shoulder_center, hip_center):
        """
        Calculate trunk angle T with vertical axis

        Args:
            shoulder_center: [x, y, z] coordinates
            hip_center: [x, y, z] coordinates

        Returns:
            Trunk angle T in degrees
        """
        # Create vertical reference point (same x, z as hip, but y offset)
        vertical_point = [hip_center[0], hip_center[1] - 1.0, hip_center[2]]

        # Calculate angle between shoulder-hip and hip-vertical
        angle = self.calculate_angle_3d(shoulder_center, hip_center, vertical_point)

        return angle

    def calculate_knee_angle(self, hip, knee, ankle):
        """
        Calculate knee angle K

        Args:
            hip, knee, ankle: [x, y, z] coordinates

        Returns:
            Knee angle K in degrees
        """
        return self.calculate_angle_3d(hip, knee, ankle)

    def calculate_chaffin_forces(self, b, h, T, K):
        """
        Calculate compression and shear forces using Chaffin's model

        Args:
            b: Distance from shoulder center to L5/S1 (meters)
            h: Distance from hand to L5/S1 (meters)
            T: Trunk angle with vertical (degrees)
            K: Knee angle (degrees)

        Returns:
            tuple: (compression_force, shear_force) in Newtons
        """
        # Convert angles to radians
        T_rad = np.radians(T)
        alpha_rad = np.radians(self.alpha)

        # Calculate moment arm 'a'
        a = h * np.sin(T_rad) + (b * np.sin(T_rad) / 2)

        # Calculate muscle force E
        numerator = (self.load_force * h + self.trunk_force * b/2) * np.sin(T_rad)
        denominator = self.e * np.sin(alpha_rad)
        E = numerator / (denominator + 1e-6)

        # Calculate compression force Fc
        Fc = E * np.cos(alpha_rad) + (self.load_force + self.trunk_force) * np.cos(T_rad)

        # Calculate shear force Fs
        Fs = E * np.sin(alpha_rad) - (self.load_force + self.trunk_force) * np.sin(T_rad)

        return abs(Fc), abs(Fs)

In [13]:
class AngleCalculator:
    """Helper class for calculating various body angles from landmarks"""

    @staticmethod
    def calculate_angle(a, b, c):
        """
        Calculate angle at point b formed by points a-b-c in 2D

        Args:
            a, b, c: Lists or arrays with [x, y] coordinates

        Returns:
            Angle in degrees
        """
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360 - angle

        return angle

    @staticmethod
    def calculate_hip_angle(shoulder, hip, knee):
        """Calculate hip angle"""
        return AngleCalculator.calculate_angle(shoulder, hip, knee)

    @staticmethod
    def calculate_knee_angle(hip, knee, ankle):
        """Calculate knee angle"""
        return AngleCalculator.calculate_angle(hip, knee, ankle)

    @staticmethod
    def calculate_ankle_angle(knee, ankle, foot):
        """Calculate ankle angle"""
        return AngleCalculator.calculate_angle(knee, ankle, foot)

In [14]:
class DeadliftStageDetector:
    """Detects the current stage of deadlift movement"""

    def __init__(self):
        self.stage = "ready"
        self.prev_stage = "ready"
        self.rep_count = 0
        self.correct_reps = 0
        self.incorrect_reps = 0
        self.current_rep_form_scores = []

    def detect_stage(self, hip_angle, knee_angle):
        """
        Detect current deadlift stage based on joint angles

        Stages:
        - ready: Starting position
        - setup: Grabbing the bar
        - lifting: Pulling the weight up
        - lockout: Standing upright
        - lowering: Returning to start

        Args:
            hip_angle: Current hip angle in degrees
            knee_angle: Current knee angle in degrees

        Returns:
            Current stage name
        """
        self.prev_stage = self.stage

        if hip_angle < DeadliftConfig.SETUP_HIP_ANGLE:
            new_stage = "setup"
        elif hip_angle < DeadliftConfig.LIFTING_HIP_ANGLE:
            new_stage = "lifting"
        elif hip_angle >= DeadliftConfig.LOCKOUT_HIP_ANGLE:
            new_stage = "lockout"
        else:
            new_stage = "transition"

        # Count reps when transitioning from lockout back to setup
        if self.prev_stage == "lockout" and new_stage in ["setup", "lifting"]:
            self.rep_count += 1

            # Determine if rep was correct based on form scores
            if len(self.current_rep_form_scores) > 0:
                avg_score = np.mean(self.current_rep_form_scores)
                if avg_score >= 70:
                    self.correct_reps += 1
                else:
                    self.incorrect_reps += 1

            self.current_rep_form_scores = []

        self.stage = new_stage
        return self.stage

    def add_form_score(self, score):
        """Add a form score for the current rep"""
        self.current_rep_form_scores.append(score)


In [15]:
class FormAnalyzer:
    """Analyzes deadlift form and provides feedback"""

    def __init__(self):
        self.feedback_messages = []
        self.form_score = 100
        self.issues = []

    def analyze_form(self, angles, forces, stage):
        """
        Analyze form based on angles, forces, and current stage

        Args:
            angles: Dictionary of joint angles
            forces: Tuple of (compression, shear) forces
            stage: Current deadlift stage

        Returns:
            Dictionary containing analysis results
        """
        self.feedback_messages = []
        self.issues = []
        self.form_score = 100

        # Analyze trunk angle
        trunk_angle = angles.get('trunk', 0)
        if stage in ["setup", "lifting"]:
            if trunk_angle < DeadliftConfig.TRUNK_ANGLE_MIN:
                self.feedback_messages.append("Back too vertical - lean forward more")
                self.issues.append("trunk_too_vertical")
                self.form_score -= 15
            elif trunk_angle > DeadliftConfig.TRUNK_ANGLE_MAX:
                self.feedback_messages.append("Back bent too far - lift chest up")
                self.issues.append("trunk_too_bent")
                self.form_score -= 20

        # Analyze knee angle
        knee_angle = angles.get('knee', 0)
        if stage == "setup":
            if knee_angle < DeadliftConfig.KNEE_ANGLE_MIN:
                self.feedback_messages.append("Knees too bent - raise hips slightly")
                self.issues.append("knees_too_bent")
                self.form_score -= 15
            elif knee_angle > DeadliftConfig.KNEE_ANGLE_MAX:
                self.feedback_messages.append("Knees too straight - squat down more")
                self.issues.append("knees_too_straight")
                self.form_score -= 15

        # Analyze forces
        compression, shear = forces
        if compression > DeadliftConfig.COMPRESSION_MAX_LIMIT:
            self.feedback_messages.append("DANGER: Excessive spinal compression!")
            self.issues.append("dangerous_compression")
            self.form_score -= 40
        elif compression > DeadliftConfig.COMPRESSION_SAFE_LIMIT:
            self.feedback_messages.append("WARNING: High spinal compression")
            self.issues.append("high_compression")
            self.form_score -= 20

        if shear > DeadliftConfig.SHEAR_MAX_LIMIT:
            self.feedback_messages.append("DANGER: Excessive shear force!")
            self.issues.append("dangerous_shear")
            self.form_score -= 40
        elif shear > DeadliftConfig.SHEAR_SAFE_LIMIT:
            self.feedback_messages.append("WARNING: High shear force")
            self.issues.append("high_shear")
            self.form_score -= 20

        # Analyze hip angle for balance
        hip_angle = angles.get('hip', 0)
        if stage == "setup" and hip_angle < DeadliftConfig.HIP_ANGLE_MIN:
            self.feedback_messages.append("Hips too low - raise them slightly")
            self.issues.append("hips_too_low")
            self.form_score -= 10

        if len(self.feedback_messages) == 0:
            self.feedback_messages.append("Good form!")

        return {
            'score': max(0, self.form_score),
            'feedback': self.feedback_messages,
            'issues': self.issues,
            'is_safe': compression < DeadliftConfig.COMPRESSION_SAFE_LIMIT and
                      shear < DeadliftConfig.SHEAR_SAFE_LIMIT
        }

In [16]:
class DataLogger:
    """Logs deadlift data for training and analysis"""

    def __init__(self, output_dir=None):
        self.output_dir = output_dir or DeadliftConfig.OUTPUT_DIR
        os.makedirs(self.output_dir, exist_ok=True)

        self.session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.landmarks_file = os.path.join(self.output_dir, f"landmarks_{self.session_id}.csv")
        self.analysis_file = os.path.join(self.output_dir, f"analysis_{self.session_id}.csv")

        # Initialize CSV files
        self._init_landmarks_csv()
        self._init_analysis_csv()

    def _init_landmarks_csv(self):
        """Initialize landmarks CSV file with headers"""
        with open(self.landmarks_file, 'w', newline='') as f:
            writer = csv.writer(f)
            header = ['timestamp', 'frame_number']
            # Add headers for all 33 landmarks (x, y, z, visibility)
            for i in range(33):
                header.extend([f'lm{i}_x', f'lm{i}_y', f'lm{i}_z', f'lm{i}_vis'])
            writer.writerow(header)

    def _init_analysis_csv(self):
        """Initialize analysis CSV file with headers"""
        with open(self.analysis_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                'timestamp', 'frame_number', 'stage', 'rep_count',
                'hip_angle', 'knee_angle', 'trunk_angle',
                'compression_force', 'shear_force',
                'form_score', 'is_safe', 'feedback'
            ])

    def log_landmarks(self, timestamp, frame_num, landmarks):
        """Log pose landmarks to CSV"""
        with open(self.landmarks_file, 'a', newline='') as f:
            writer = csv.writer(f)
            row = [timestamp, frame_num]
            for lm in landmarks.landmark:
                row.extend([lm.x, lm.y, lm.z, lm.visibility])
            writer.writerow(row)

    def log_analysis(self, timestamp, frame_num, analysis_data):
        """Log analysis results to CSV"""
        with open(self.analysis_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                timestamp, frame_num,
                analysis_data.get('stage', ''),
                analysis_data.get('rep_count', 0),
                analysis_data.get('hip_angle', 0),
                analysis_data.get('knee_angle', 0),
                analysis_data.get('trunk_angle', 0),
                analysis_data.get('compression', 0),
                analysis_data.get('shear', 0),
                analysis_data.get('form_score', 0),
                analysis_data.get('is_safe', False),
                '; '.join(analysis_data.get('feedback', []))
            ])

In [17]:
class DeadliftAnalyzer:
    """
    Main class that integrates all components for real-time
    deadlift form analysis using MediaPipe
    """

    def __init__(self, body_weight=70, load_weight=20, height=1.75,
                 video_source="WhatsApp Video 2025-11-13 um 06.56.03_9c52e65a", save_data=True):
        """
        Initialize the deadlift analyzer

        Args:
            body_weight: User's body weight in kg
            load_weight: Barbell weight in kg
            height: User's height in meters
            video_source: Camera index or video file path
            save_data: Whether to save analysis data
        """
        # Initialize MediaPipe Pose
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles

        self.pose = self.mp_pose.Pose(
            min_detection_confidence=DeadliftConfig.MIN_DETECTION_CONFIDENCE,
            min_tracking_confidence=DeadliftConfig.MIN_TRACKING_CONFIDENCE,
            model_complexity=DeadliftConfig.MODEL_COMPLEXITY
        )

        # Initialize components
        self.bio_calc = BiomechanicalCalculator(body_weight, load_weight, height)
        self.angle_calc = AngleCalculator()
        self.stage_detector = DeadliftStageDetector()
        self.form_analyzer = FormAnalyzer()

        # Initialize data logger if saving is enabled
        self.save_data = save_data
        if save_data:
            self.data_logger = DataLogger()

        # Initialize video capture
        self.video_source = video_source
        self.cap = None

        # Frame counter
        self.frame_count = 0

        # FPS calculation
        self.fps_queue = deque(maxlen=30)
        self.prev_time = 0

    def get_landmark_coords(self, landmarks, landmark_name):
        """
        Extract 3D coordinates for a specific landmark

        Args:
            landmarks: MediaPipe pose landmarks
            landmark_name: Name of the landmark (from DeadliftConfig.LANDMARKS)

        Returns:
            [x, y, z] coordinates, or None if not found
        """
        idx = DeadliftConfig.LANDMARKS.get(landmark_name)
        if idx is None:
            return None

        lm = landmarks.landmark[idx]
        return [lm.x, lm.y, lm.z]

    def get_landmark_coords_2d(self, landmarks, landmark_name, frame_width, frame_height):
        """
        Extract 2D pixel coordinates for a specific landmark

        Args:
            landmarks: MediaPipe pose landmarks
            landmark_name: Name of the landmark
            frame_width: Width of the video frame
            frame_height: Height of the video frame

        Returns:
            [x, y] pixel coordinates
        """
        coords_3d = self.get_landmark_coords(landmarks, landmark_name)
        if coords_3d is None:
            return None

        return [int(coords_3d[0] * frame_width), int(coords_3d[1] * frame_height)]

    def extract_key_points(self, landmarks):
        """
        Extract all key points needed for biomechanical analysis

        Args:
            landmarks: MediaPipe pose landmarks

        Returns:
            Dictionary of key points
        """
        key_points = {}

        # Extract individual landmarks
        for name in ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip',
                     'left_knee', 'right_knee', 'left_ankle', 'right_ankle',
                     'left_wrist', 'right_wrist']:
            key_points[name] = self.get_landmark_coords(landmarks, name)

        # Calculate center points
        if key_points['left_shoulder'] and key_points['right_shoulder']:
            key_points['shoulder_center'] = [
                (key_points['left_shoulder'][0] + key_points['right_shoulder'][0]) / 2,
                (key_points['left_shoulder'][1] + key_points['right_shoulder'][1]) / 2,
                (key_points['left_shoulder'][2] + key_points['right_shoulder'][2]) / 2
            ]

        if key_points['left_hip'] and key_points['right_hip']:
            key_points['hip_center'] = [
                (key_points['left_hip'][0] + key_points['right_hip'][0]) / 2,
                (key_points['left_hip'][1] + key_points['right_hip'][1]) / 2,
                (key_points['left_hip'][2] + key_points['right_hip'][2]) / 2
            ]

        return key_points

    def calculate_biomechanics(self, key_points):
        """
        Calculate biomechanical parameters and forces

        Args:
            key_points: Dictionary of extracted landmarks

        Returns:
            Dictionary containing angles and forces
        """
        results = {
            'angles': {},
            'forces': (0, 0),
            'distances': {}
        }

        # Use right side for analysis (can be modified for left or bilateral)
        shoulder = key_points.get('right_shoulder')
        hip = key_points.get('right_hip')
        knee = key_points.get('right_knee')
        ankle = key_points.get('right_ankle')
        shoulder_center = key_points.get('shoulder_center')
        hip_center = key_points.get('hip_center')
        hand = key_points.get('right_wrist')

        if not all([shoulder, hip, knee, ankle, shoulder_center, hip_center, hand]):
            return results

        # Calculate angles (2D for display)
        hip_angle = self.angle_calc.calculate_hip_angle(
            shoulder[:2], hip[:2], knee[:2]
        )
        knee_angle = self.angle_calc.calculate_knee_angle(
            hip[:2], knee[:2], ankle[:2]
        )

        # Calculate trunk angle (3D)
        trunk_angle = self.bio_calc.calculate_trunk_angle(
            shoulder_center, hip_center
        )

        results['angles'] = {
            'hip': hip_angle,
            'knee': knee_angle,
            'trunk': trunk_angle
        }

        # Calculate distances for Chaffin's model
        # b: distance from shoulder center to hip center (L5/S1 approximation)
        b = self.bio_calc.calculate_distance(shoulder_center, hip_center)

        # h: distance from hand to hip center (load distance)
        h = self.bio_calc.calculate_distance(hand, hip_center)

        results['distances'] = {
            'b': b,
            'h': h
        }

        # Calculate forces using Chaffin's model
        try:
            compression, shear = self.bio_calc.calculate_chaffin_forces(
                b, h, trunk_angle, knee_angle
            )
            results['forces'] = (compression, shear)
        except Exception as e:
            print(f"Error calculating forces: {e}")
            results['forces'] = (0, 0)

        return results

    def draw_info_panel(self, frame, stage, rep_count, angles, forces, form_analysis):
        """
        Draw information panel on the frame

        Args:
            frame: Video frame
            stage: Current deadlift stage
            rep_count: Current rep count
            angles: Dictionary of angles
            forces: Tuple of (compression, shear) forces
            form_analysis: Form analysis results
        """
        h, w = frame.shape[:2]

        # Semi-transparent overlay
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (w - 10, 250), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)

        # Text parameters
        font = cv2.FONT_HERSHEY_SIMPLEX
        y_offset = 40
        line_height = 30

        # Stage and rep count
        cv2.putText(frame, f"Stage: {stage.upper()}", (20, y_offset),
                   font, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Reps: {rep_count}", (20, y_offset + line_height),
                   font, 0.7, (255, 255, 255), 2)

        y_offset += line_height * 2

        # Angles
        cv2.putText(frame, "ANGLES:", (20, y_offset),
                   font, 0.6, (100, 255, 100), 2)
        y_offset += line_height

        hip_angle = angles.get('hip', 0)
        knee_angle = angles.get('knee', 0)
        trunk_angle = angles.get('trunk', 0)

        cv2.putText(frame, f"Hip: {hip_angle:.1f} deg", (30, y_offset),
                   font, 0.5, (255, 255, 255), 1)
        y_offset += line_height
        cv2.putText(frame, f"Knee: {knee_angle:.1f} deg", (30, y_offset),
                   font, 0.5, (255, 255, 255), 1)
        y_offset += line_height
        cv2.putText(frame, f"Trunk: {trunk_angle:.1f} deg", (30, y_offset),
                   font, 0.5, (255, 255, 255), 1)

        y_offset += line_height

        # Forces
        compression, shear = forces
        cv2.putText(frame, "FORCES (L5/S1):", (20, y_offset),
                   font, 0.6, (100, 255, 255), 2)
        y_offset += line_height

        # Color code based on safety
        comp_color = (0, 255, 0) if compression < DeadliftConfig.COMPRESSION_SAFE_LIMIT else \
                     (0, 255, 255) if compression < DeadliftConfig.COMPRESSION_MAX_LIMIT else (0, 0, 255)
        shear_color = (0, 255, 0) if shear < DeadliftConfig.SHEAR_SAFE_LIMIT else \
                      (0, 255, 255) if shear < DeadliftConfig.SHEAR_MAX_LIMIT else (0, 0, 255)

        cv2.putText(frame, f"Compression: {compression:.0f} N", (30, y_offset),
                   font, 0.5, comp_color, 1)
        y_offset += line_height
        cv2.putText(frame, f"Shear: {shear:.0f} N", (30, y_offset),
                   font, 0.5, shear_color, 1)

        # Form score
        y_offset = h - 150
        score = form_analysis.get('score', 0)
        score_color = (0, 255, 0) if score >= 80 else \
                     (0, 255, 255) if score >= 60 else (0, 0, 255)

        cv2.putText(frame, f"Form Score: {score}/100", (20, y_offset),
                   font, 0.7, score_color, 2)

        # Feedback messages
        y_offset += line_height
        feedback = form_analysis.get('feedback', [])
        for msg in feedback[:3]:  # Show max 3 messages
            cv2.putText(frame, msg, (20, y_offset),
                       font, 0.5, (255, 255, 255), 1)
            y_offset += line_height

    def process_frame(self, frame):
        """
        Process a single frame for pose detection and analysis

        Args:
            frame: Input video frame

        Returns:
            Processed frame with overlays
        """
        self.frame_count += 1

        # Convert to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process with MediaPipe
        results = self.pose.process(frame_rgb)

        # Default values
        stage = "waiting"
        angles = {}
        forces = (0, 0)
        form_analysis = {'score': 0, 'feedback': ['No person detected'], 'is_safe': True}

        if results.pose_landmarks:
            # Draw pose landmarks
            self.mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                self.mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=self.mp_drawing_styles.get_default_pose_landmarks_style()
            )

            # Extract key points
            key_points = self.extract_key_points(results.pose_landmarks)

            # Calculate biomechanics
            biomech_results = self.calculate_biomechanics(key_points)
            angles = biomech_results['angles']
            forces = biomech_results['forces']

            # Detect stage
            if 'hip' in angles and 'knee' in angles:
                stage = self.stage_detector.detect_stage(angles['hip'], angles['knee'])

            # Analyze form
            form_analysis = self.form_analyzer.analyze_form(angles, forces, stage)
            self.stage_detector.add_form_score(form_analysis['score'])

            # Save data if enabled
            if self.save_data:
                timestamp = time.time()
                self.data_logger.log_landmarks(timestamp, self.frame_count, results.pose_landmarks)

                analysis_data = {
                    'stage': stage,
                    'rep_count': self.stage_detector.rep_count,
                    'hip_angle': angles.get('hip', 0),
                    'knee_angle': angles.get('knee', 0),
                    'trunk_angle': angles.get('trunk', 0),
                    'compression': forces[0],
                    'shear': forces[1],
                    'form_score': form_analysis['score'],
                    'is_safe': form_analysis['is_safe'],
                    'feedback': form_analysis['feedback']
                }
                self.data_logger.log_analysis(timestamp, self.frame_count, analysis_data)

        # Draw information panel
        self.draw_info_panel(
            frame,
            stage,
            self.stage_detector.rep_count,
            angles,
            forces,
            form_analysis
        )

        # Calculate and display FPS
        current_time = time.time()
        fps = 1 / (current_time - self.prev_time + 1e-6)
        self.prev_time = current_time
        self.fps_queue.append(fps)
        avg_fps = np.mean(self.fps_queue)

        cv2.putText(frame, f"FPS: {avg_fps:.1f}", (frame.shape[1] - 150, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        return frame

    def run(self):
        """Run the deadlift analyzer on video stream"""
        # Open video capture
        self.cap = cv2.VideoCapture(self.video_source)

        if not self.cap.isOpened():
            print("Error: Could not open video source")
            return

        # Set camera properties
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, DeadliftConfig.FRAME_WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, DeadliftConfig.FRAME_HEIGHT)

        # Video writer for saving
        video_writer = None
        if DeadliftConfig.SAVE_VIDEO and self.save_data:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_path = os.path.join(
                DeadliftConfig.OUTPUT_DIR,
                f"deadlift_{self.data_logger.session_id}.mp4"
            )
            video_writer = cv2.VideoWriter(
                output_path, fourcc, 20.0,
                (DeadliftConfig.FRAME_WIDTH, DeadliftConfig.FRAME_HEIGHT)
            )

        print("Deadlift Analyzer Started")
        print("Press 'q' to quit, 'r' to reset rep counter")

        while True:
            ret, frame = self.cap.read()

            if not ret:
                print("Failed to grab frame")
                break

            # Process frame
            processed_frame = self.process_frame(frame)

            # Save frame if video writer is initialized
            if video_writer:
                video_writer.write(processed_frame)

            # Display frame
            cv2.imshow('Deadlift Form Analyzer', processed_frame)

            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                self.stage_detector.rep_count = 0
                self.stage_detector.correct_reps = 0
                self.stage_detector.incorrect_reps = 0
                print("Rep counter reset")

        # Cleanup
        self.cap.release()
        if video_writer:
            video_writer.release()
        cv2.destroyAllWindows()

        # Print session summary
        print(f"\nSession Summary:")
        print(f"Total Frames: {self.frame_count}")
        print(f"Total Reps: {self.stage_detector.rep_count}")
        print(f"Correct Reps: {self.stage_detector.correct_reps}")
        print(f"Incorrect Reps: {self.stage_detector.incorrect_reps}")
        if self.save_data:
            print(f"Data saved to: {DeadliftConfig.OUTPUT_DIR}")

# ============================================================================
# MAIN ENTRY POINT
# ============================================================================
if __name__ == "__main__":
    # User configuration
    print("=== Deadlift Form Analyzer ===")
    print("Using default parameters:")
    print("Body weight: 70 kg")
    print("Load weight: 20 kg")
    print("Height: 1.75 m")
    print("\nStarting analyzer...")

    # Create and run analyzer
    analyzer = DeadliftAnalyzer(
        body_weight=70,
        load_weight=20,
        height=1.75,
        video_source=0,  # 0 for webcam, or path to video file
        save_data=True
    )

=== Deadlift Form Analyzer ===
Using default parameters:
Body weight: 70 kg
Load weight: 20 kg
Height: 1.75 m

Starting analyzer...


In [18]:
analyzer.run()

Error: Could not open video source
